In [1]:
from kaggle_environments import evaluate, make, utils

import math
import random
import numpy as np
    
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD
from kaggle_environments import evaluate, make, utils, agent
import pickle

Loading environment football failed: No module named 'gfootball'


# Var init for training

In [2]:
DISCOUNT = 0.99

DRAW = 0.3
WIN = 1
LOSE = -1
ERROR = -5

EPSILON = 1  # not a constant, going to be decayed
EPSILON_DECAY = 0.9993
MIN_EPSILON = 0.001

In [3]:
class ConnectX:
    
    def __init__(self, pair=[None, "random"], config={"rows": 6, "columns": 7, "inarow": 4}, N_STEPS=4):
        self.env = make("connectx", config, debug=True)
        self.config = config
        self.set_pair(pair)
        self.N_STEPS = N_STEPS
        
    def set_pair(self, pair):
        self.pair = pair
    
    def switch_pair(self):
        self.pair = self.pair[::-1]
    
    def change_reward(self, reward, done):
        if done:
            if reward is None: #Error - column has already filled
                reward = ERROR
            elif reward == 1:
                reward = WIN
            elif reward == -1:
                reward = LOSE
            elif reward == 0:
                reward = DRAW
        else:
            reward = -1/(self.config['rows'] * self.config['columns'])
        return reward
    
    # Calculates score if agent drops piece in selected column
    def score_move(self, grid, col, mark, config, nsteps):
        next_grid = self.drop_piece(grid, col, mark, config)
        score = self.minimax(next_grid, nsteps-1, -math.inf, math.inf, False, mark, config)
        return score
    
    # Helper function for minimax: checks if agent or opponent has four in a row in the window
    def is_terminal_window(self, window, config):
        return window.count(1) == config['inarow'] or window.count(2) == config['inarow']

    # Helper function for score_move: gets board at next step if agent drops piece in selected column
    def drop_piece(self, grid, col, mark, config):
        next_grid = grid.copy()
        for row in range(config['rows']-1, -1, -1):
            if next_grid[row][col] == 0:
                break
        next_grid[row][col] = mark
        return next_grid

    # Helper function for minimax: checks if game has ended
    def is_terminal_node(self, grid, config):
        # Check for draw 
        if list(grid[0, :]).count(0) == 0:
            return True
        # Check for win: horizontal, vertical, or diagonal
        # horizontal 
        for row in range(config['rows']):
            for col in range(config['rows']-(config['inarow']-1)):
                window = list(grid[row, col:col+config['inarow']])
                if self.is_terminal_window(window, config):
                    return True
        # vertical
        for row in range(config['rows']-(config['inarow']-1)):
            for col in range(config['columns']):
                window = list(grid[row:row+config['inarow'], col])
                if self.is_terminal_window(window, config):
                    return True
        # positive diagonal
        for row in range(config['rows']-(config['inarow']-1)):
            for col in range(config['columns']-(config['inarow']-1)):
                window = list(grid[range(row, row+config['inarow']), range(col, col+config['inarow'])])
                if self.is_terminal_window(window, config):
                    return True
        # negative diagonal
        for row in range(config['inarow']-1, config['rows']):
            for col in range(config['columns']-(config['inarow']-1)):
                window = list(grid[range(row, row-config['inarow'], -1), range(col, col+config['inarow'])])
                if self.is_terminal_window(window, config):
                    return True
        return False
    
    # Helper function for score_move: calculates value of heuristic for grid
    def get_heuristic(self, grid, mark, config):
        
        A = 10000
        B = 2
        C = 1
        D = -1
        E = -5
        
        num_twos = self.count_windows(grid, 2, mark, config)
        num_threes = self.count_windows(grid, 3, mark, config)
        num_fours = self.count_windows(grid, 4, mark, config)
        num_twos_opp = self.count_windows(grid, 2, mark%2+1, config)
        num_threes_opp = self.count_windows(grid, 3, mark%2+1, config)
        score = A*num_fours + B*num_threes + C*num_twos + D*num_twos_opp + E*num_threes_opp
        return score
    
    # Minimax implementation
    def minimax(self, node, depth, alpha, beta, maximizingPlayer, mark, config):
        is_terminal = self.is_terminal_node(node, config)
        valid_moves = [c for c in range(config['rows']) if node[0][c] == 0]
        
        if depth == 0 or is_terminal:
            return self.get_heuristic(node, mark, config)
        
        if maximizingPlayer:
            value = -np.Inf
            try:
                best_col = random.choice(valid_moves)
            except:
                print('valid moves ', valid_moves)
            for col in valid_moves:
                child = self.drop_piece(node, col, mark, config)
                new_score = self.minimax(child, depth-1, alpha, beta, False, mark, config)
                if new_score > value:
                    value = new_score
                    best_col = col
                alpha = max(alpha,value)
                if alpha >= beta:
                        break
            return value
        
        else:
            value = np.Inf
            try:
                best_col = random.choice(valid_moves)
            except:
                print('valid moves ', valid_moves)
            for col in valid_moves:
                child = self.drop_piece(node, col, mark%2+1, config)
                new_score = self.minimax(child, depth-1, alpha, beta, True, mark, config)
                
                if new_score < value:
                    value = new_score
                    best_col = col
                beta = min(beta,value)
                if beta <=alpha:
                    break
            
            return value

    # Helper function for get_heuristic: checks if window satisfies heuristic conditions
    def check_window(self, window, num_discs, piece, config):
        return (window.count(piece) == num_discs and window.count(0) == config['inarow']-num_discs)

    # Helper function for get_heuristic: counts number of windows satisfying specified heuristic conditions
    def count_windows(self, grid, num_discs, piece, config):
        num_windows = 0
        # horizontal
        for row in range(config['rows']):
            for col in range(config['columns']-(config['inarow']-1)):
                window = list(grid[row, col:col+config['inarow']])
                if self.check_window(window, num_discs, piece, config):
                    num_windows += 1
        # vertical
        for row in range(config['rows']-(config['inarow']-1)):
            for col in range(config['columns']):
                window = list(grid[row:row+config['inarow'], col])
                if self.check_window(window, num_discs, piece, config):
                    num_windows += 1
        # positive diagonal
        for row in range(config['rows']-(config['inarow']-1)):
            for col in range(config['columns']-(config['inarow']-1)):
                window = list(grid[range(row, row+config['inarow']), range(col, col+config['inarow'])])
                if self.check_window(window, num_discs, piece, config):
                    num_windows += 1
        # negative diagonal
        for row in range(config['inarow']-1, config['rows']):
            for col in range(config['columns']-(config['inarow']-1)):
                window = list(grid[range(row, row-config['inarow'], -1), range(col, col+config['inarow'])])
                if self.check_window(window, num_discs, piece, config):
                    num_windows += 1
        return num_windows
    
    def play_game(self, model, render=False):
        global EPSILON
        trainer = self.env.train(self.pair)
        observation = trainer.reset()
        steps = []

        while not self.env.done:
            current_state = get_board(observation, self.config)
            
            # Get list of valid moves
            valid_moves = [c for c in range(self.config["columns"]) if observation['board'][c] == 0]  
                
            # Convert the board to a 2D grid
            grid = np.asarray(observation['board']).reshape(self.config["rows"], self.config['columns'])
            
            #Calculate the nbr of free moves to see if we use the minimax or the model
            nbr_free_move = sum([list(grid[x,:]).count(0) for x in range(0,6)])
            
            # Predict the move with DQN
            current_qs = model.predict(current_state[None,:])[0]
            if np.random.random() > EPSILON:
                action = int(np.argmax(current_qs))
            else:
                action = np.random.randint(0, self.config["columns"] - 1)    

            #If valid move then we choose the move from the model
            if action in valid_moves:
                current_qs = current_qs
                action = action
            # Else we use the heuristic
            else:
                # Use the heuristic to assign a score to each possible board in the next step
                scores = dict(zip(valid_moves, [self.score_move(grid, col, observation['mark'], self.config, self.N_STEPS) for col in valid_moves]))
                # Get a list of columns (moves) that maximize the heuristic
                max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
                # Set all probability to 0
                current_qs = [0, 0, 0, 0, 0, 0, 0]
                # Update proba of the good column to 1
                current_qs[random.choice(max_cols)] = 1
                action = random.choice(max_cols)
            
            observation, reward, done, info = trainer.step(action)
            reward = self.change_reward(reward, done)
            
            steps.append((current_state, current_qs, action, reward, done))
            
        if render:
            self.env.render(mode="ipython", width=500, height=500)
            
        # Decay epsilon
        if EPSILON > MIN_EPSILON:
            EPSILON *= EPSILON_DECAY
            EPSILON = max(MIN_EPSILON, EPSILON)
            
        return (reward, steps)

In [4]:
class DQN:
    def __init__(self):
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        layers = [
             Conv2D(64, 3, activation='relu', kernel_initializer=initializer, input_shape=(6,7,1)),
             Conv2D(64, 3, activation='relu', kernel_initializer=initializer),
             Flatten(),
             Dense(50, activation='relu', kernel_initializer=initializer),
             Dense(50, activation='relu', kernel_initializer=initializer),
             Dropout(0.1),
             Dense(7)
        ]
        model = tf.keras.Sequential(layers)
        # clipvalue and clipnorm needs to prevent gradient explosion
        model.compile(loss="mse", optimizer=Adam(lr=0.0005, clipvalue = 1., clipnorm = 1.), metrics=['accuracy'])
        
        self.model = model
        
    def train(self, steps, epochs=10):
        X = []
        y = []
        
        er_before = 0
        for index in range(len(steps)-1, -1 , -1):
            current_state, current_qs, action, reward, done = steps[index]

            # If not a terminal state, get new q from future states, otherwise set it to 0
            # almost like with Q Learning, but we use just part of equation here
            if not done:
                max_future_q = np.max(steps[index + 1][1])
                new_q = reward + DISCOUNT * max_future_q
            else:
                new_q = reward

            # Update Q value for given state
            current_qs[action] = new_q
            current_qs[current_qs!=action] -= reward

            # And append to our training data
            X.append(current_state)
            y.append(current_qs)
            
            # Add symetrical state
            X.append(current_state[:, ::-1, :])
            y.append(current_qs[::-1])
            
            
        self.model.fit(np.array(X), np.array(y), batch_size=64, epochs = epochs)
    
    def predict(self, X):
        return self.model.predict(X)
    
    def save_model(self):
        self.model.save("P8_KAGGLE/DQN_Minimax2")
        
    def return_model(self):
        return self.model
    
class Trainer:
    def __init__(self, game, model):
        self.game = game
        self.model = model
    
    def _print_statistics(self, rewards):
        rewards = np.array(rewards)
        print("Wins:", (rewards == WIN).sum())
        print("Loses:", (rewards == LOSE).sum())
        print("Errors:", (rewards == ERROR).sum())

    def train(self, num_of_games=1000, every=100, epochs=50, switch=True):
        games = []
        rewards = []
        
        for i in range(1, num_of_games + 1):
            reward, steps = self.game.play_game(self.model)
            games = games + steps
            rewards.append(reward)
            
            if i % every == 0:
                if switch:
                    self.game.switch_pair()
                self._print_statistics(rewards)
                self.model.train(games, epochs)
                games = []
                rewards = []
        
        if len(games) > 200:
            self._print_statistics(rewards)
            self.model.train(games, epochs)
            
def get_board(observation, configuration):
    rows = configuration['rows']
    columns = configuration['columns']

    board = np.array(observation['board']).reshape((rows,columns,1))
    new_board = np.zeros_like(board)

    mark = observation["mark"]
    new_board[board == mark] = 1
    new_board[(board != mark) & (board != 0)] = 2
    return new_board / 2

In [5]:
model = DQN()
game = ConnectX(config={"rows": 6, "columns": 7, "inarow": 4})
trainer = Trainer(game, model)

In [6]:
EPSILON = 1.0
game.set_pair(['negamax', None])
trainer.train(num_of_games=10000, every=100, epochs=30, switch=True)

Wins: 0
Loses: 100
Errors: 0
Epoch 1/30
21/21 [==============================] - 1s 2ms/step - loss: 0.4336 - accuracy: 0.2110
Epoch 2/30
21/21 [==============================] - 0s 2ms/step - loss: 0.3480 - accuracy: 0.2425
Epoch 3/30
21/21 [==============================] - 0s 2ms/step - loss: 0.3407 - accuracy: 0.2330
Epoch 4/30
21/21 [==============================] - 0s 2ms/step - loss: 0.3265 - accuracy: 0.2589
Epoch 5/30
21/21 [==============================] - 0s 2ms/step - loss: 0.3107 - accuracy: 0.2448
Epoch 6/30
21/21 [==============================] - 0s 2ms/step - loss: 0.3106 - accuracy: 0.2879
Epoch 7/30
21/21 [==============================] - 0s 2ms/step - loss: 0.3021 - accuracy: 0.2561
Epoch 8/30
21/21 [==============================] - 0s 2ms/step - loss: 0.2917 - accuracy: 0.2884
Epoch 9/30
21/21 [==============================] - 0s 2ms/step - loss: 0.2875 - accuracy: 0.2967
Epoch 10/30
21/21 [==============================] - 0s 2ms/step - loss: 0.2780 - accurac

Epoch 14/30
27/27 [==============================] - 0s 3ms/step - loss: 0.6216 - accuracy: 0.4620
Epoch 15/30
27/27 [==============================] - 0s 2ms/step - loss: 0.6187 - accuracy: 0.4521
Epoch 16/30
27/27 [==============================] - 0s 2ms/step - loss: 0.6138 - accuracy: 0.4574
Epoch 17/30
27/27 [==============================] - 0s 2ms/step - loss: 0.6045 - accuracy: 0.4632
Epoch 18/30
27/27 [==============================] - 0s 2ms/step - loss: 0.5966 - accuracy: 0.4568
Epoch 19/30
27/27 [==============================] - 0s 2ms/step - loss: 0.5897 - accuracy: 0.4714
Epoch 20/30
27/27 [==============================] - 0s 2ms/step - loss: 0.5919 - accuracy: 0.4813
Epoch 21/30
27/27 [==============================] - 0s 2ms/step - loss: 0.5836 - accuracy: 0.4673
Epoch 22/30
27/27 [==============================] - 0s 2ms/step - loss: 0.5784 - accuracy: 0.4696
Epoch 23/30
27/27 [==============================] - 0s 2ms/step - loss: 0.5792 - accuracy: 0.4796
Epoch 24/3

26/26 [==============================] - 0s 2ms/step - loss: 0.5884 - accuracy: 0.5862
Epoch 28/30
26/26 [==============================] - 0s 3ms/step - loss: 0.5644 - accuracy: 0.6025
Epoch 29/30
26/26 [==============================] - 0s 3ms/step - loss: 0.5650 - accuracy: 0.6043
Epoch 30/30
26/26 [==============================] - 0s 3ms/step - loss: 0.5683 - accuracy: 0.6068
Wins: 4
Loses: 92
Errors: 0
Epoch 1/30
27/27 [==============================] - 0s 2ms/step - loss: 1.0771 - accuracy: 0.4883
Epoch 2/30
27/27 [==============================] - 0s 2ms/step - loss: 0.9748 - accuracy: 0.5100
Epoch 3/30
27/27 [==============================] - 0s 2ms/step - loss: 0.9328 - accuracy: 0.5117
Epoch 4/30
27/27 [==============================] - 0s 2ms/step - loss: 0.9031 - accuracy: 0.5188
Epoch 5/30
27/27 [==============================] - 0s 2ms/step - loss: 0.8894 - accuracy: 0.5223
Epoch 6/30
27/27 [==============================] - 0s 2ms/step - loss: 0.8593 - accuracy: 0.5164


23/23 [==============================] - 0s 2ms/step - loss: 0.5709 - accuracy: 0.5394
Epoch 11/30
23/23 [==============================] - 0s 2ms/step - loss: 0.5716 - accuracy: 0.5352
Epoch 12/30
23/23 [==============================] - 0s 2ms/step - loss: 0.5628 - accuracy: 0.5527
Epoch 13/30
23/23 [==============================] - 0s 2ms/step - loss: 0.5602 - accuracy: 0.5429
Epoch 14/30
23/23 [==============================] - 0s 2ms/step - loss: 0.5513 - accuracy: 0.5506
Epoch 15/30
23/23 [==============================] - 0s 2ms/step - loss: 0.5551 - accuracy: 0.5359
Epoch 16/30
23/23 [==============================] - 0s 2ms/step - loss: 0.5469 - accuracy: 0.5640
Epoch 17/30
23/23 [==============================] - 0s 2ms/step - loss: 0.5497 - accuracy: 0.5513
Epoch 18/30
23/23 [==============================] - 0s 2ms/step - loss: 0.5401 - accuracy: 0.5506
Epoch 19/30
23/23 [==============================] - 0s 2ms/step - loss: 0.5399 - accuracy: 0.5556
Epoch 20/30
23/23 [===

31/31 [==============================] - 0s 2ms/step - loss: 0.3682 - accuracy: 0.5260
Epoch 23/30
31/31 [==============================] - 0s 2ms/step - loss: 0.3597 - accuracy: 0.5240
Epoch 24/30
31/31 [==============================] - 0s 2ms/step - loss: 0.3660 - accuracy: 0.5158
Epoch 25/30
31/31 [==============================] - 0s 2ms/step - loss: 0.3707 - accuracy: 0.5174
Epoch 26/30
31/31 [==============================] - 0s 2ms/step - loss: 0.3595 - accuracy: 0.5194
Epoch 27/30
31/31 [==============================] - 0s 2ms/step - loss: 0.3539 - accuracy: 0.5204
Epoch 28/30
31/31 [==============================] - 0s 2ms/step - loss: 0.3542 - accuracy: 0.5220
Epoch 29/30
31/31 [==============================] - 0s 2ms/step - loss: 0.3640 - accuracy: 0.5240
Epoch 30/30
31/31 [==============================] - 0s 2ms/step - loss: 0.3498 - accuracy: 0.5179
Wins: 8
Loses: 91
Errors: 0
Epoch 1/30
28/28 [==============================] - 0s 2ms/step - loss: 0.6434 - accuracy: 0.

...

20/20 [==============================] - 0s 3ms/step - loss: 0.9549 - accuracy: 0.4185
Epoch 2/30
20/20 [==============================] - 0s 2ms/step - loss: 0.8953 - accuracy: 0.3805
Epoch 3/30
20/20 [==============================] - 0s 2ms/step - loss: 0.8499 - accuracy: 0.3924
Epoch 4/30
20/20 [==============================] - 0s 2ms/step - loss: 0.8192 - accuracy: 0.3600
Epoch 5/30
20/20 [==============================] - 0s 2ms/step - loss: 0.8085 - accuracy: 0.3932
Epoch 6/30
20/20 [==============================] - 0s 2ms/step - loss: 0.7927 - accuracy: 0.3884
Epoch 7/30
20/20 [==============================] - 0s 3ms/step - loss: 0.7868 - accuracy: 0.3869
Epoch 8/30
20/20 [==============================] - 0s 2ms/step - loss: 0.7683 - accuracy: 0.3932
Epoch 9/30
20/20 [==============================] - 0s 3ms/step - loss: 0.7666 - accuracy: 0.4082
Epoch 10/30
20/20 [==============================] - 0s 2ms/step - loss: 0.7598 - accuracy: 0.3774
Epoch 11/30
20/20 [===========

19/19 [==============================] - 0s 2ms/step - loss: 0.6400 - accuracy: 0.2725
Epoch 16/30
19/19 [==============================] - 0s 2ms/step - loss: 0.6597 - accuracy: 0.2758
Epoch 17/30
19/19 [==============================] - 0s 2ms/step - loss: 0.6448 - accuracy: 0.2742
Epoch 18/30
19/19 [==============================] - 0s 2ms/step - loss: 0.6502 - accuracy: 0.2708
Epoch 19/30
19/19 [==============================] - 0s 2ms/step - loss: 0.6426 - accuracy: 0.2733
Epoch 20/30
19/19 [==============================] - 0s 2ms/step - loss: 0.6441 - accuracy: 0.2675
Epoch 21/30
19/19 [==============================] - 0s 2ms/step - loss: 0.6366 - accuracy: 0.2742
Epoch 22/30
19/19 [==============================] - 0s 3ms/step - loss: 0.6287 - accuracy: 0.2867
Epoch 23/30
19/19 [==============================] - 0s 3ms/step - loss: 0.6340 - accuracy: 0.2767
Epoch 24/30
19/19 [==============================] - 0s 2ms/step - loss: 0.6252 - accuracy: 0.2733
Epoch 25/30
19/19 [===

33/33 [==============================] - 0s 3ms/step - loss: 0.4207 - accuracy: 0.3150
Epoch 30/30
33/33 [==============================] - 0s 2ms/step - loss: 0.4463 - accuracy: 0.3058
Wins: 1
Loses: 98
Errors: 0
Epoch 1/30
20/20 [==============================] - 0s 2ms/step - loss: 1.0096 - accuracy: 0.4463
Epoch 2/30
20/20 [==============================] - 0s 2ms/step - loss: 0.9606 - accuracy: 0.4373
Epoch 3/30
20/20 [==============================] - 0s 2ms/step - loss: 0.9202 - accuracy: 0.4178
Epoch 4/30
20/20 [==============================] - 0s 2ms/step - loss: 0.8858 - accuracy: 0.4446
Epoch 5/30
20/20 [==============================] - 0s 2ms/step - loss: 0.8986 - accuracy: 0.4169
Epoch 6/30
20/20 [==============================] - 0s 2ms/step - loss: 0.8579 - accuracy: 0.4332
Epoch 7/30
20/20 [==============================] - 0s 2ms/step - loss: 0.8554 - accuracy: 0.4047
Epoch 8/30
20/20 [==============================] - 0s 2ms/step - loss: 0.8366 - accuracy: 0.4186
Ep

37/37 [==============================] - 0s 2ms/step - loss: 0.3532 - accuracy: 0.2855
Epoch 14/30
37/37 [==============================] - 0s 3ms/step - loss: 0.3396 - accuracy: 0.2800
Epoch 15/30
37/37 [==============================] - 0s 3ms/step - loss: 0.3432 - accuracy: 0.2851
Epoch 16/30
37/37 [==============================] - 0s 3ms/step - loss: 0.3365 - accuracy: 0.2796
Epoch 17/30
37/37 [==============================] - 0s 3ms/step - loss: 0.3419 - accuracy: 0.2757
Epoch 18/30
37/37 [==============================] - 0s 3ms/step - loss: 0.3413 - accuracy: 0.2723
Epoch 19/30
37/37 [==============================] - 0s 3ms/step - loss: 0.3312 - accuracy: 0.2885
Epoch 20/30
37/37 [==============================] - 0s 3ms/step - loss: 0.3491 - accuracy: 0.2847
Epoch 21/30
37/37 [==============================] - 0s 3ms/step - loss: 0.3320 - accuracy: 0.2766
Epoch 22/30
37/37 [==============================] - 0s 2ms/step - loss: 0.3305 - accuracy: 0.2809
Epoch 23/30
37/37 [===

28/28 [==============================] - 0s 2ms/step - loss: 0.4710 - accuracy: 0.3241
Epoch 27/30
28/28 [==============================] - 0s 2ms/step - loss: 0.4645 - accuracy: 0.3281
Epoch 28/30
28/28 [==============================] - 0s 3ms/step - loss: 0.4731 - accuracy: 0.3235
Epoch 29/30
28/28 [==============================] - 0s 2ms/step - loss: 0.4706 - accuracy: 0.3460
Epoch 30/30
28/28 [==============================] - 0s 2ms/step - loss: 0.4664 - accuracy: 0.3339
Wins: 3
Loses: 96
Errors: 0
Epoch 1/30
21/21 [==============================] - 0s 2ms/step - loss: 0.8580 - accuracy: 0.3804
Epoch 2/30
21/21 [==============================] - 0s 2ms/step - loss: 0.8076 - accuracy: 0.3300
Epoch 3/30
21/21 [==============================] - 0s 2ms/step - loss: 0.7711 - accuracy: 0.3051
Epoch 4/30
21/21 [==============================] - 0s 2ms/step - loss: 0.7590 - accuracy: 0.3222
Epoch 5/30
21/21 [==============================] - 0s 2ms/step - loss: 0.7432 - accuracy: 0.3090

In [11]:
model.save_model()

INFO:tensorflow:Assets written to: P8_KAGGLE/DQN_Minimax\assets


In [12]:
model1 = tf.keras.models.load_model('P8_KAGGLE/DQN_Minimax')

In [13]:
def agent(observation, configuration):
    
    N_STEPS = 4
    
    # Calculates score if agent drops piece in selected column
    def score_move(grid, col, mark, config, nsteps):
        next_grid = drop_piece(grid, col, mark, config)
        score = minimax(next_grid, nsteps-1, -math.inf, math.inf, False, mark, config)
        return score
    
    # Helper function for minimax: checks if agent or opponent has four in a row in the window
    def is_terminal_window(window, config):
        return window.count(1) == config['inarow'] or window.count(2) == config['inarow']

    # Helper function for score_move: gets board at next step if agent drops piece in selected column
    def drop_piece(grid, col, mark, config):
        next_grid = grid.copy()
        for row in range(config['rows']-1, -1, -1):
            if next_grid[row][col] == 0:
                break
        next_grid[row][col] = mark
        return next_grid

    # Helper function for minimax: checks if game has ended
    def is_terminal_node(grid, config):
        # Check for draw 
        if list(grid[0, :]).count(0) == 0:
            return True
        # Check for win: horizontal, vertical, or diagonal
        # horizontal 
        for row in range(config['rows']):
            for col in range(config['rows']-(config['inarow']-1)):
                window = list(grid[row, col:col+config['inarow']])
                if is_terminal_window(window, config):
                    return True
        # vertical
        for row in range(config['rows']-(config['inarow']-1)):
            for col in range(config['columns']):
                window = list(grid[row:row+config['inarow'], col])
                if is_terminal_window(window, config):
                    return True
        # positive diagonal
        for row in range(config['rows']-(config['inarow']-1)):
            for col in range(config['columns']-(config['inarow']-1)):
                window = list(grid[range(row, row+config['inarow']), range(col, col+config['inarow'])])
                if is_terminal_window(window, config):
                    return True
        # negative diagonal
        for row in range(config['inarow']-1, config['rows']):
            for col in range(config['columns']-(config['inarow']-1)):
                window = list(grid[range(row, row-config['inarow'], -1), range(col, col+config['inarow'])])
                if is_terminal_window(window, config):
                    return True
        return False
    
    # Helper function for score_move: calculates value of heuristic for grid
    def get_heuristic(grid, mark, config):
        
        A = 10000
        B = 2
        C = 1
        D = -1
        E = -5
        
        num_twos = count_windows(grid, 2, mark, config)
        num_threes = count_windows(grid, 3, mark, config)
        num_fours = count_windows(grid, 4, mark, config)
        num_twos_opp = count_windows(grid, 2, mark%2+1, config)
        num_threes_opp = count_windows(grid, 3, mark%2+1, config)
        score = A*num_fours + B*num_threes + C*num_twos + D*num_twos_opp + E*num_threes_opp
        return score
    
    # Minimax implementation
    def minimax(node, depth, alpha, beta, maximizingPlayer, mark, config):
        is_terminal = is_terminal_node(node, config)
        valid_moves = [c for c in range(config['rows']) if node[0][c] == 0]
        
        if depth == 0 or is_terminal:
            return get_heuristic(node, mark, config)
        
        if maximizingPlayer:
            value = -np.Inf
            try:
                best_col = random.choice(valid_moves)
            except:
                print('valid moves ', valid_moves)
            for col in valid_moves:
                child = drop_piece(node, col, mark, config)
                new_score = minimax(child, depth-1, alpha, beta, False, mark, config)
                if new_score > value:
                    value = new_score
                    best_col = col
                alpha = max(alpha,value)
                if alpha >= beta:
                        break
            return value
        
        else:
            value = np.Inf
            try:
                best_col = random.choice(valid_moves)
            except:
                print('valid moves ', valid_moves)
            for col in valid_moves:
                child = drop_piece(node, col, mark%2+1, config)
                new_score = minimax(child, depth-1, alpha, beta, True, mark, config)
                
                if new_score < value:
                    value = new_score
                    best_col = col
                beta = min(beta,value)
                if beta <=alpha:
                    break
            
            return value

    # Helper function for get_heuristic: checks if window satisfies heuristic conditions
    def check_window(window, num_discs, piece, config):
        return (window.count(piece) == num_discs and window.count(0) == config['inarow']-num_discs)

    # Helper function for get_heuristic: counts number of windows satisfying specified heuristic conditions
    def count_windows(grid, num_discs, piece, config):
        num_windows = 0
        # horizontal
        for row in range(config['rows']):
            for col in range(config['columns']-(config['inarow']-1)):
                window = list(grid[row, col:col+config['inarow']])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # vertical
        for row in range(config['rows']-(config['inarow']-1)):
            for col in range(config['columns']):
                window = list(grid[row:row+config['inarow'], col])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # positive diagonal
        for row in range(config['rows']-(config['inarow']-1)):
            for col in range(config['columns']-(config['inarow']-1)):
                window = list(grid[range(row, row+config['inarow']), range(col, col+config['inarow'])])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # negative diagonal
        for row in range(config['inarow']-1, config['rows']):
            for col in range(config['columns']-(config['inarow']-1)):
                window = list(grid[range(row, row-config['inarow'], -1), range(col, col+config['inarow'])])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        return num_windows    
    
    current_state = get_board(observation, configuration)
            
    # Get list of valid moves
    valid_moves = [c for c in range(configuration["columns"]) if observation['board'][c] == 0]        
    # Convert the board to a 2D grid
    grid = np.asarray(observation['board']).reshape(configuration["rows"], configuration['columns'])
            
    #Calculate the nbr of free moves to see if we use the minimax or the model
    nbr_free_move = sum([list(grid[x,:]).count(0) for x in range(0,6)])
            
    # Might not be the most pytonic way of doing it
    if nbr_free_move < 20:
        # Predict the move with DQN
        current_qs = model1.predict(current_state[None,:])[0]  

        #If valid move then we choose the move from the model
        if action in valid_moves:
            action = action
        # Else we use the heuristic
        else:
            # Use the heuristic to assign a score to each possible board in the next step
            scores = dict(zip(valid_moves, [score_move(grid, col, observation['mark'], configuration, N_STEPS) for col in valid_moves]))
            # Get a list of columns (moves) that maximize the heuristic
            max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
            action = random.choice(max_cols)
                    
    elif 20 <= nbr_free_move <= 27:
        # Use the heuristic to assign a score to each possible board in the next step
        scores = dict(zip(valid_moves, [score_move(grid, col, observation['mark'], configuration, N_STEPS) for col in valid_moves]))
        # Get a list of columns (moves) that maximize the heuristic
        max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
        action = random.choice(max_cols)
             
    else:
        #Changing the number of steps for minimax as the game is almost ended
        N_STEPS = 5
        # Use the heuristic to assign a score to each possible board in the next step
        scores = dict(zip(valid_moves, [score_move(grid, col, observation['mark'], configuration, N_STEPS) for col in valid_moves]))
        # Get a list of columns (moves) that maximize the heuristic
        max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
        action = random.choice(max_cols)    
        
    return action

In [16]:
env = make("connectx", debug=True)
env.play([agent, None])